In [ ]:
# CENO M-PL 1.0.3 - Computational Enzyme Network Optimization

**Description**:  
CENO M-PL (Computational Enzyme Network Optimization for Molecular Protein-Ligand Systems) is a Python-based tool designed to automate molecular dynamics simulations (MDS) for protein-ligand interactions. It aims to optimize enzyme networks using automated workflows to streamline protein-ligand system analysis. This tool supports error handling, folder management, and is highly customizable for various protein-ligand complexes.

**Author**:  
Cruz Francisco Osuna Aguirre  
*Email*: cruzzosuna@ciencias.unam.mx  
*Alternate Email*: cruzzosuna2003@gmail.com

**Last Update**:  
October 26, 2024

## Version History

- **v1.0.3**  
  - Adaptation of functions for protein-ligand systems.

- **v1.0.2**  
  - Improved folder management and error handling.

- **v1.0.1**  
  - Operation verified. Added functions for individual and global result analysis.

- **v1.0.0**  
  - Initial version with all steps automated. Pending verification of full operation.

---

For citations or further information, please refer to the included documentation or contact the author via email.



In [ ]:
import os
print(os.getcwd())


In [ ]:
# MIT License
# 
# Copyright (c) 2024 Cruz Francisco Osuna Aguirre
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to use
# the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# 1. The above copyright notice and this permission notice shall be included
# in all copies or substantial portions of the Software.
# 
# 2. The use of the Software for commercial purposes is prohibited without
# the express consent of the author.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

def display_license():
    license_text = """
    MIT License

    Copyright (c) [YEAR] Cruz Francisco Osuna Aguirre

    Permission is hereby granted, free of charge, to any person obtaining a copy
    of this software and associated documentation files (the "Software"), to use
    the Software without restriction, including without limitation the rights
    to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
    copies of the Software, and to permit persons to whom the Software is
    furnished to do so, subject to the following conditions:

    1. The above copyright notice and this permission notice shall be included
    in all copies or substantial portions of the Software.

    2. The use of the Software for commercial purposes is prohibited without
    the express consent of the author.

    THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
    IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
    FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
    AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
    LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
    OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
    SOFTWARE.
    """
    print(license_text)

# Call the function to display the license at the start of your program
if __name__ == "__main__":
    display_license()
    # Your main code starts here


In [ ]:
#Protein preparation

import os
import subprocess
import logging
from datetime import datetime

# Configurar el registro de logs
log_filename = f"protein_preparation_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(filename=log_filename, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Paso 1: Preparación de la Proteína
def prepare_protein(protein_pdb_path, protein_name):
    try:
        if not os.path.exists(protein_pdb_path):
            raise FileNotFoundError(f"El archivo {protein_pdb_path} no existe.")

        # Crear el script de Chimera para procesar la proteína
        script_content = f"""open {protein_pdb_path}
delete :UNK.A
delete :UNL.A
addh
addcharge
write format pdb 0 {protein_name}_REC.pdb
"""
        script_filename = "chimera_script.cmd"
        with open(script_filename, "w") as script_file:
            script_file.write(script_content)

        # Ejecutar Chimera con el archivo de script
        chimera_command = f"/opt/UCSF/Chimera64-1.18/bin/chimera --nogui {script_filename}"
        result = subprocess.run(chimera_command, shell=True, capture_output=True, text=True)
        
        if result.returncode != 0:
            logging.error(f"Error al ejecutar Chimera para {protein_name}. Salida: {result.stderr}")
            raise subprocess.CalledProcessError(result.returncode, chimera_command, output=result.stdout, stderr=result.stderr)

        output_pdb = f"{protein_name}_REC.pdb"
        if not os.path.exists(output_pdb):
            raise FileNotFoundError(f"{output_pdb} no fue creado. Verifica el comando de Chimera.")

        logging.info(f"Archivo {output_pdb} creado exitosamente.")
    except FileNotFoundError as fnf_error:
        logging.error(fnf_error)
        raise
    except subprocess.CalledProcessError as process_error:
        logging.error(f"Error en prepare_protein para {protein_name}: {process_error.stderr}")
        raise
    except Exception as e:
        logging.error(f"Error inesperado en prepare_protein para {protein_name}: {e}")
        raise

# Paso 2: Crear directorio para guardar los archivos de la proteína
def create_output_directory(protein_name):
    try:
        output_dir = f"{protein_name}_MDS"
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        return output_dir
    except OSError as os_error:
        logging.error(f"Error al crear el directorio {output_dir} para {protein_name}: {os_error}")
        raise
    except Exception as e:
        logging.error(f"Error inesperado al crear el directorio para {protein_name}: {e}")
        raise

# Ejecutar la preparación de proteínas para todas las proteínas en la carpeta proteins_pdb
def prepare_proteins():
    protein_dir = "proteins_pdb"

    # Verificar que el directorio exista
    if not os.path.exists(protein_dir):
        logging.error(f"La carpeta {protein_dir} no existe. Verifica la ruta.")
        return

    # Obtener todos los archivos PDB en el directorio
    protein_files = [f for f in os.listdir(protein_dir) if f.endswith(".pdb")]

    if not protein_files:
        logging.warning("No se encontraron archivos PDB en la carpeta proteins_pdb.")
        return

    for protein_pdb in protein_files:
        try:
            protein_pdb_path = os.path.join(protein_dir, protein_pdb)
            if not os.path.exists(protein_pdb_path):
                logging.warning(f"Archivo PDB {protein_pdb_path} no encontrado.")
                continue

            protein_name = os.path.splitext(protein_pdb)[0]
            logging.info(f"Iniciando preparación para {protein_name}")

            # Crear directorio de salida para cada proteína
            output_dir = create_output_directory(protein_name)
            prepare_protein(protein_pdb_path, protein_name)

            # Mover el archivo preparado a su carpeta correspondiente
            prepared_pdb = f"{protein_name}_REC.pdb"
            final_path = os.path.join(output_dir, prepared_pdb)

            if os.path.exists(prepared_pdb):
                os.rename(prepared_pdb, final_path)
                logging.info(f"Archivo {prepared_pdb} movido a {final_path}.")
            else:
                logging.error(f"El archivo preparado {prepared_pdb} no se encuentra para {protein_name}.")

        except Exception as e:
            logging.error(f"Preparación falló para {protein_name}: {e}")
            continue  # Continuar con la siguiente proteína

# Correr la preparación de proteínas
if __name__ == "__main__":
    try:
        prepare_proteins()
    except Exception as e:
        logging.critical(f"Error crítico durante la ejecución: {e}")



In [ ]:
#Copy of configuration files to protein folders

import os
import shutil
import logging

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Lista de archivos requeridos a copiar desde la carpeta 'configuration'
required_files = ["LIG.itp", "LIG.pdb", "ions.mdp", "EM.mdp", "NVT.mdp", "NPT.mdp", "MD.mdp"]

# Directorio de configuración donde se encuentran los archivos requeridos
config_dir = "configuration"

# Función para obtener todas las carpetas generadas por el anterior script (que terminen en "_MDS")
def get_protein_folders():
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Función para verificar si todos los archivos requeridos existen en el directorio de configuración
def check_required_files():
    missing_files = [file for file in required_files if not os.path.exists(os.path.join(config_dir, file))]
    
    if missing_files:
        for file in missing_files:
            logging.error(f"El archivo requerido {file} no se encontró en {config_dir}.")
        return False
    return True

# Función para copiar los archivos requeridos a cada carpeta de proteínas
def copy_required_files_to_protein_folders():
    # Verificar si todos los archivos requeridos existen
    if not check_required_files():
        logging.error("No se pudieron encontrar todos los archivos requeridos. Proceso abortado.")
        return
    
    # Obtener las carpetas de proteínas generadas automáticamente
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'. Verifica si se han creado correctamente.")
        return

    # Copiar los archivos a cada carpeta de proteína
    for folder in protein_folders:
        try:
            # Verificar permisos de escritura en la carpeta de destino
            if not os.access(folder, os.W_OK):
                logging.error(f"No se tienen permisos de escritura en la carpeta {folder}.")
                continue

            for file in required_files:
                source_file_path = os.path.join(config_dir, file)
                destination = os.path.join(folder, file)

                # Intentar copiar el archivo
                shutil.copy(source_file_path, destination)

                # Verificar si el archivo fue copiado correctamente
                if not os.path.exists(destination):
                    logging.error(f"Error al copiar {file} a {folder}. El archivo no se encuentra en la carpeta de destino.")
                else:
                    logging.info(f"Archivo {file} copiado exitosamente a {folder}.")
        except FileNotFoundError as fnf_error:
            logging.error(f"Archivo no encontrado durante la copia: {fnf_error}")
        except PermissionError as perm_error:
            logging.error(f"Permisos insuficientes al copiar archivos a {folder}: {perm_error}")
        except Exception as e:
            logging.error(f"Error inesperado al copiar archivos a la carpeta {folder}: {e}")
        finally:
            logging.info(f"Proceso de copia para la carpeta {folder} completado.")

# Ejecutar la función de copia de archivos
if __name__ == "__main__":
    copy_required_files_to_protein_folders()


In [ ]:
#Create topol.top of protein

import os
import subprocess
import logging

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Comando para activar GROMACS
gromacs_source_command = ". /usr/local/gromacs/bin/GMXRC"

# Función para obtener todas las carpetas de proteínas que terminen en "_MDS"
def get_protein_folders():
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Función para ejecutar el comando gmx pdb2gmx dentro de cada carpeta
def run_gromacs_in_protein_folders():
    # Obtener las carpetas de proteínas dinámicamente
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'. Verifica si se han creado correctamente.")
        return

    # Ejecutar comandos GROMACS en cada carpeta de proteínas
    for folder in protein_folders:
        try:
            if not os.path.exists(folder):
                logging.warning(f"La carpeta {folder} no existe. Saltando...")
                continue
            
            # Verificar que podemos cambiar al directorio de la proteína
            original_dir = os.getcwd()
            try:
                os.chdir(folder)
            except OSError as e:
                logging.error(f"No se pudo acceder a la carpeta {folder}: {e}")
                continue

            # Extraer el nombre de la proteína eliminando el sufijo "_MDS"
            protein_name = folder.replace("_MDS", "")

            # Verificar que el archivo REC.pdb existe
            pdb_file = f"{protein_name}_REC.pdb"
            if not os.path.exists(pdb_file):
                logging.warning(f"El archivo {pdb_file} no se encontró en la carpeta {folder}. Saltando...")
                os.chdir(original_dir)
                continue

            # Ejecutar el comando para activar GROMACS
            try:
                subprocess.run(gromacs_source_command, shell=True, check=True)
            except subprocess.CalledProcessError as e:
                logging.error(f"Error al activar GROMACS en {folder}: {e}")
                os.chdir(original_dir)
                continue

            # Comando gmx pdb2gmx con las opciones 8 (CHARMM27) y 1 (TIP3P)
            pdb2gmx_command = f"echo '8\n1\n' | gmx pdb2gmx -f {pdb_file} -ignh"
            try:
                subprocess.run(pdb2gmx_command, shell=True, check=True)
                logging.info(f"Comando gmx pdb2gmx ejecutado exitosamente en {folder}.")
            except subprocess.CalledProcessError as e:
                logging.error(f"Error al ejecutar gmx pdb2gmx en {folder}: {e}")
                os.chdir(original_dir)
                continue

            # Volver al directorio original
            os.chdir(original_dir)

        except Exception as e:
            logging.error(f"Error inesperado en {folder}: {e}")
            # Asegurarse de volver al directorio original si ocurre un error
            os.chdir(original_dir)
            continue

# Ejecutar el script
if __name__ == "__main__":
    run_gromacs_in_protein_folders()


In [ ]:
# .gro ligand file generation

import os
import subprocess
import logging

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Función para detectar todas las carpetas que terminen en "_MDS"
def get_protein_folders():
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Comando para generar el archivo .gro del ligando
def generate_gro_file(input_pdb, output_gro):
    try:
        if not os.path.exists(input_pdb):
            raise FileNotFoundError(f"El archivo {input_pdb} no existe.")
        
        # Ejecutar gmx editconf para generar el archivo .gro
        subprocess.run(f"gmx editconf -f {input_pdb} -o {output_gro}", shell=True, check=True)
        logging.info(f"Archivo .gro generado para {input_pdb}.")
    except FileNotFoundError as e:
        logging.error(e)
        raise
    except subprocess.CalledProcessError as e:
        logging.error(f"Error al generar {output_gro}: {e}")
        raise
    except Exception as e:
        logging.error(f"Error inesperado al generar {output_gro}: {e}")
        raise

# Función para modificar conf.gro con los datos del ligando
def modify_conf_gro(conf_gro, add_gro):
    try:
        # Verificar que los archivos existen antes de proceder
        if not os.path.exists(conf_gro):
            raise FileNotFoundError(f"El archivo {conf_gro} no existe.")
        if not os.path.exists(add_gro):
            raise FileNotFoundError(f"El archivo {add_gro} no existe.")
        
        # Leer el contenido de conf.gro
        with open(conf_gro, "r") as conf_file:
            conf_lines = conf_file.readlines()

        # Leer el contenido del archivo .gro a insertar (LIG.gro)
        with open(add_gro, "r") as add_file:
            add_lines = add_file.readlines()[2:-1]  # Omitir las dos primeras líneas y la última

        # Modificar conf.gro agregando las líneas desde el tercer elemento del archivo .gro (LIG)
        conf_lines = conf_lines[:-1] + add_lines + [conf_lines[-1]]  # Agregar antes de la última línea

        # Actualizar el número de átomos en la segunda línea de conf.gro
        atom_count = len(conf_lines) - 3
        conf_lines[1] = f"{atom_count}\n"

        # Guardar el archivo modificado
        with open(conf_gro, "w") as conf_file:
            conf_file.writelines(conf_lines)

        logging.info(f"conf.gro modificado exitosamente con el contenido de {add_gro}.")
    except FileNotFoundError as e:
        logging.error(e)
        raise
    except Exception as e:
        logging.error(f"Error al modificar conf.gro con {add_gro}: {e}")
        raise

# Función para ejecutar todo el proceso dentro de una carpeta de proteína
def automate_topology_inclusion_in_folder(folder):
    original_dir = os.getcwd()  # Guardar el directorio actual
    try:
        # Cambiar al directorio de la proteína
        os.chdir(folder)
        
        # Verificar la existencia de los archivos iniciales
        if not os.path.exists("LIG.pdb"):
            logging.warning(f"Archivo LIG.pdb no encontrado en {folder}. Saltando...")
            return
        
        if not os.path.exists("conf.gro"):
            logging.warning(f"El archivo conf.gro no se encontró en {folder}. Saltando...")
            return

        # Generar el archivo .gro para LIG
        generate_gro_file("LIG.pdb", "LIG.gro")

        # Modificar conf.gro para incluir los datos de LIG.gro
        modify_conf_gro("conf.gro", "LIG.gro")

        logging.info(f"Proceso completado en {folder}.")

    except Exception as e:
        logging.error(f"Error en el proceso automatizado en {folder}: {e}")
    finally:
        os.chdir(original_dir)  # Asegurarse de volver al directorio original

# Ejecutar el proceso en todas las carpetas de proteínas
def run_automate_in_all_folders():
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'. Verifica si se han creado correctamente.")
        return

    for folder in protein_folders:
        logging.info(f"Iniciando proceso en {folder}.")
        automate_topology_inclusion_in_folder(folder)

# Ejecutar el script
if __name__ == "__main__":
    run_automate_in_all_folders()


In [ ]:
#topol.top edition
import os
import logging

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Función para detectar todas las carpetas que terminen en "_MDS"
def get_protein_folders():
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Función para editar el archivo topol.top
def edit_topol_top(topol_file):
    try:
        if not os.path.exists(topol_file):
            logging.error(f"El archivo {topol_file} no existe.")
            return

        # Leer el contenido del archivo topol.top
        with open(topol_file, "r") as file:
            lines = file.readlines()

        # Buscar la línea de 'forcefield.itp' y 'Protein_chain_A'
        forcefield_index = -1
        protein_chain_index = -1
        molecules_section_index = -1

        for i, line in enumerate(lines):
            if '#include "charmm27.ff/forcefield.itp"' in line:
                forcefield_index = i + 1  # Insertar justo debajo de esta línea
            if "[ molecules ]" in line:
                molecules_section_index = i + 1  # Para confirmar que estamos en la sección correcta
            if "Protein_chain_A" in line:
                protein_chain_index = i + 1  # Insertar justo debajo de esta línea

        if forcefield_index == -1 or protein_chain_index == -1 or molecules_section_index == -1:
            logging.error("No se encontraron las secciones esperadas en topol.top. Verifica el archivo.")
            return

        # Modificación en la sección de forcefield, eliminando referencias al cofactor (COF)
        topology_to_add = (
            "\n; Include ligand topology\n"
            '#include "LIG.itp"\n'
        )
        lines.insert(forcefield_index, topology_to_add)

        # Modificación en la sección de moléculas justo debajo de Protein_chain_A
        molecules_to_add = (
            "LIG                 1\n"
        )
        lines.insert(protein_chain_index + 1, molecules_to_add)

        # Guardar el archivo modificado
        with open(topol_file, "w") as file:
            file.writelines(lines)

        logging.info(f"El archivo {topol_file} ha sido editado correctamente.")

    except FileNotFoundError as e:
        logging.error(f"Archivo no encontrado: {e}")
    except Exception as e:
        logging.error(f"Error al editar {topol_file}: {e}")
        raise

# Función para ejecutar el proceso en todas las carpetas de proteínas
def run_edit_topol_in_all_folders():
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'. Verifica si se han creado correctamente.")
        return

    for folder in protein_folders:
        original_dir = os.getcwd()  # Guardar el directorio original
        try:
            # Verificar si la carpeta existe
            if not os.path.exists(folder):
                logging.warning(f"La carpeta {folder} no existe. Saltando...")
                continue

            # Cambiar al directorio de la proteína
            os.chdir(folder)
            topol_file = "topol.top"

            logging.info(f"Iniciando edición de {topol_file} en {folder}.")
            
            # Ejecutar la edición del archivo topol.top
            edit_topol_top(topol_file)

        except Exception as e:
            logging.error(f"Error en el proceso automatizado en {folder}: {e}")
        finally:
            # Volver al directorio principal, incluso si ocurre un error
            os.chdir(original_dir)

# Ejecutar el script
if __name__ == "__main__":
    run_edit_topol_in_all_folders()


In [ ]:
#Solvatacion

import os
import subprocess
import logging

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Función para detectar todas las carpetas que terminen en "_MDS"
def get_protein_folders():
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Función para ejecutar comandos de solvatación en cada carpeta
def run_solvation_in_folder(folder):
    try:
        # Verificar si los archivos necesarios existen
        conf_file = "conf.gro"
        topol_file = "topol.top"
        if not os.path.exists(conf_file) or not os.path.exists(topol_file):
            logging.warning(f"Archivos {conf_file} o {topol_file} faltan en {folder}. Saltando...")
            return

        # Comando gmx editconf para generar box.gro
        editconf_command = f"gmx editconf -f {conf_file} -d 1.0 -bt dodecahedron -o box.gro"
        subprocess.run(editconf_command, shell=True, check=True)
        logging.info(f"Comando editconf ejecutado correctamente en {folder}.")

        # Verificar si box.gro fue creado
        if not os.path.exists("box.gro"):
            logging.error(f"El archivo box.gro no se creó correctamente en {folder}.")
            return

        # Comando gmx solvate para generar box_sol.gro
        solvate_command = f"gmx solvate -cp box.gro -cs spc216.gro -p {topol_file} -o box_sol.gro"
        subprocess.run(solvate_command, shell=True, check=True)
        logging.info(f"Comando solvate ejecutado correctamente en {folder}.")

        # Verificar si box_sol.gro fue creado
        if not os.path.exists("box_sol.gro"):
            logging.error(f"El archivo box_sol.gro no se creó correctamente en {folder}.")
            return

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al ejecutar el comando en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado en {folder}: {e}")

# Función para ejecutar el proceso en todas las carpetas de proteínas
def run_solvation_in_all_folders():
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'. Verifica si se han creado correctamente.")
        return

    for folder in protein_folders:
        original_dir = os.getcwd()  # Guardar el directorio original
        try:
            # Verificar si la carpeta existe
            if not os.path.exists(folder):
                logging.warning(f"La carpeta {folder} no existe. Saltando...")
                continue

            # Cambiar al directorio de la carpeta de la proteína
            os.chdir(folder)
            logging.info(f"Iniciando solvatación en {folder}.")

            # Ejecutar los comandos de solvatación
            run_solvation_in_folder(folder)

        except Exception as e:
            logging.error(f"Error en el proceso de solvatación en {folder}: {e}")
        finally:
            # Asegurar volver al directorio principal
            os.chdir(original_dir)

# Ejecutar el script
if __name__ == "__main__":
    run_solvation_in_all_folders()


In [ ]:
# Adición de iones

import os
import subprocess
import logging

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Función para detectar todas las carpetas que terminen en "_MDS"
def get_protein_folders():
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Función para ejecutar los comandos de adición de iones en cada carpeta
def run_ion_addition_in_folder(folder):
    try:
        # Verificar si los archivos necesarios existen
        ions_mdp_file = "ions.mdp"
        box_sol_file = "box_sol.gro"
        topol_file = "topol.top"
        if not os.path.exists(ions_mdp_file) or not os.path.exists(box_sol_file) or not os.path.exists(topol_file):
            logging.warning(f"Archivos {ions_mdp_file}, {box_sol_file} o {topol_file} faltan en {folder}. Saltando...")
            return

        # Comando gmx grompp para generar ION.tpr con -maxwarn 2
        grompp_command = f"gmx grompp -f {ions_mdp_file} -c {box_sol_file} -p {topol_file} -o ION.tpr -maxwarn 2"
        subprocess.run(grompp_command, shell=True, check=True)
        logging.info(f"Comando grompp ejecutado correctamente en {folder}.")

        # Verificar si ION.tpr fue creado
        if not os.path.exists("ION.tpr"):
            logging.error(f"El archivo ION.tpr no se creó correctamente en {folder}.")
            return

        # Comando gmx genion para añadir iones
        genion_command = f"echo 15 | gmx genion -s ION.tpr -p {topol_file} -conc 0.1 -neutral -o box_sol_ion.gro"
        subprocess.run(genion_command, shell=True, check=True)
        logging.info(f"Comando genion ejecutado correctamente en {folder}.")

        # Verificar si box_sol_ion.gro fue creado
        if not os.path.exists("box_sol_ion.gro"):
            logging.error(f"El archivo box_sol_ion.gro no se creó correctamente en {folder}.")
            return

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al ejecutar el comando en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado en {folder}: {e}")

# Función para ejecutar el proceso en todas las carpetas de proteínas
def run_ion_addition_in_all_folders():
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'. Verifica si se han creado correctamente.")
        return

    for folder in protein_folders:
        original_dir = os.getcwd()  # Guardar el directorio original
        try:
            # Verificar si la carpeta existe
            if not os.path.exists(folder):
                logging.warning(f"La carpeta {folder} no existe. Saltando...")
                continue

            # Cambiar al directorio de la carpeta de la proteína
            os.chdir(folder)
            logging.info(f"Iniciando adición de iones en {folder}.")

            # Ejecutar los comandos de adición de iones
            run_ion_addition_in_folder(folder)

        except Exception as e:
            logging.error(f"Error en el proceso de adición de iones en {folder}: {e}")
        finally:
            # Volver al directorio principal, incluso si ocurre un error
            os.chdir(original_dir)

# Ejecutar el script
if __name__ == "__main__":
    run_ion_addition_in_all_folders()


In [ ]:
# Minimización de Energía (Energy Minimization) para cada proteína con verificación opcional y almacenamiento en carpeta 'analisis'

import os
import subprocess
import logging

# Configurar el registro de logs con archivo de log
logging.basicConfig(
    filename='energy_minimization.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Función para detectar todas las carpetas que terminen en "_MDS"
def get_protein_folders():
    current_dir = os.getcwd()
    return [folder for folder in os.listdir(current_dir) if os.path.isdir(folder) and folder.endswith("_MDS")]

# Comando para ejecutar grompp y generar el archivo EM.tpr
def run_energy_minimization_grompp(folder):
    try:
        # Verificar si los archivos necesarios existen
        em_mdp_file = "EM.mdp"
        box_sol_ion_file = "box_sol_ion.gro"
        topol_file = "topol.top"

        missing_files = [f for f in [em_mdp_file, box_sol_ion_file, topol_file] if not os.path.exists(f)]
        if missing_files:
            logging.warning(f"Archivos {', '.join(missing_files)} faltan en {folder}. Saltando...")
            return

        # Comando grompp para preparar la simulación de minimización de energía
        grompp_command = f"gmx grompp -f {em_mdp_file} -c {box_sol_ion_file} -p {topol_file} -o EM.tpr"
        result = subprocess.run(grompp_command, shell=True, capture_output=True, text=True)

        if result.returncode != 0:
            logging.error(f"Error al ejecutar grompp en {folder}: {result.stderr}")
            return
        else:
            logging.info(f"Comando grompp ejecutado correctamente en {folder}.")

        # Verificar si EM.tpr fue creado
        if not os.path.exists("EM.tpr"):
            logging.error(f"El archivo EM.tpr no se creó correctamente en {folder}.")
            return

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al ejecutar grompp en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado al ejecutar grompp en {folder}: {e}")

# Comando para ejecutar mdrun y realizar la minimización de energía
def run_energy_minimization_mdrun(folder):
    try:
        # Verificar si el archivo EM.tpr existe
        if not os.path.exists("EM.tpr"):
            logging.warning(f"El archivo EM.tpr no existe en {folder}. No se puede ejecutar mdrun.")
            return

        # Comando mdrun para realizar la minimización de energía
        mdrun_command = "gmx mdrun -v -deffnm EM"
        result = subprocess.run(mdrun_command, shell=True, capture_output=True, text=True)

        if result.returncode != 0:
            logging.error(f"Error al ejecutar mdrun en {folder}: {result.stderr}")
            return
        else:
            logging.info(f"Minimización de energía completada exitosamente en {folder}.")

        # Verificar si EM.gro fue creado
        if not os.path.exists("EM.gro"):
            logging.error(f"El archivo EM.gro no se creó correctamente en {folder}.")
            return

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al ejecutar mdrun en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado al ejecutar mdrun en {folder}: {e}")

# Función para verificar la minimización de energía y guardar los resultados en la carpeta 'analisis'
def verify_energy_minimization(folder):
    try:
        # Verificar si el archivo EM.edr existe
        edr_file = "EM.edr"
        if not os.path.exists(edr_file):
            logging.warning(f"El archivo {edr_file} no se encontró en {folder}. Saltando verificación...")
            return

        # Crear la carpeta 'analisis' si no existe
        analysis_dir = "analisis"
        if not os.path.exists(analysis_dir):
            os.makedirs(analysis_dir)
            logging.info(f"Directorio 'analisis' creado en {folder}.")

        # Comando gmx energy para generar el archivo potential.xvg sin visualización
        energy_command = f"echo 10 | gmx energy -f {edr_file} -o potential.xvg"
        result = subprocess.run(energy_command, shell=True, capture_output=True, text=True)

        if result.returncode != 0:
            logging.error(f"Error al ejecutar gmx energy en {folder}: {result.stderr}")
            return
        else:
            logging.info(f"Archivo potential.xvg generado correctamente en {folder}.")

        # Verificar si potential.xvg fue creado
        if not os.path.exists("potential.xvg"):
            logging.error(f"El archivo potential.xvg no se creó correctamente en {folder}.")
            return

        # Mover el archivo potential.xvg al directorio 'analisis'
        destination = os.path.join(analysis_dir, "potential.xvg")
        try:
            os.rename("potential.xvg", destination)
            logging.info(f"Archivo potential.xvg movido a {destination}.")
        except Exception as e:
            logging.error(f"Error al mover potential.xvg a {destination}: {e}")

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al ejecutar gmx energy en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado en {folder}: {e}")

# Función para ejecutar el proceso completo de minimización de energía en una carpeta de proteína
def process_minimization(folder, verify):
    original_dir = os.getcwd()
    try:
        # Cambiar al directorio de la carpeta de la proteína
        os.chdir(folder)
        logging.info(f"Iniciando minimización de energía en {folder}.")

        # Ejecutar los comandos de GROMACS
        run_energy_minimization_grompp(folder)
        run_energy_minimization_mdrun(folder)

        # Verificación opcional de la minimización de energía
        if verify:
            verify_energy_minimization(folder)

    except Exception as e:
        logging.error(f"Error en el proceso de minimización de energía en {folder}: {e}")
    finally:
        # Asegurarse de volver al directorio original, incluso si ocurre un error
        os.chdir(original_dir)

# Función para ejecutar el proceso en todas las carpetas de proteínas
def run_minimization_in_all_folders(verify=False):
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'. Verifica si se han creado correctamente.")
        return

    for folder in protein_folders:
        if not os.path.exists(folder):
            logging.warning(f"La carpeta {folder} no existe. Saltando...")
            continue

        # Ejecutar el proceso de minimización en la carpeta
        process_minimization(folder, verify)

# Ejecutar el script
if __name__ == "__main__":
    run_minimization_in_all_folders(verify=False)  # Cambia verify a True si deseas verificar automáticamente



In [ ]:
# Script automatizado para la restricción del ligando en GROMACS

import os
import subprocess
import logging
from tqdm import tqdm

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Función para detectar todas las carpetas que terminen en "_MDS"
def get_protein_folders():
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Función para ejecutar make_ndx y genrestr para el ligando
def apply_ligand_restriction(folder):
    try:
        # Verificar si los archivos necesarios existen
        lig_gro_file = "LIG.gro"
        topol_file = "topol.top"
        
        if not os.path.exists(lig_gro_file) or not os.path.exists(topol_file):
            logging.warning(f"Archivos {lig_gro_file} o {topol_file} faltan en {folder}. Saltando...")
            return

        # Comando gmx make_ndx para crear el índice del ligando
        make_ndx_command = f"gmx make_ndx -f {lig_gro_file} -o index_LIG.ndx"
        process = subprocess.Popen(make_ndx_command, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
        process.communicate(input=b"0 & ! a H*\nq\n")  # Excluye átomos de hidrógeno
        process.wait()

        # Verificar si index_LIG.ndx fue creado
        if not os.path.exists("index_LIG.ndx"):
            logging.error(f"El archivo index_LIG.ndx no se creó correctamente en {folder}.")
            return

        logging.info(f"Índice del ligando creado correctamente en {folder}.")

        # Comando gmx genrestr para generar el archivo de restricciones de posición del ligando
        genrestr_command = f"gmx genrestr -f {lig_gro_file} -n index_LIG.ndx -o posre_LIG.itp -fc 1000 1000 1000"
        process = subprocess.Popen(genrestr_command, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
        process.communicate(input=b"2\n")  # Selección del grupo 2 (LIG)
        process.wait()

        # Verificar si posre_LIG.itp fue creado
        if not os.path.exists("posre_LIG.itp"):
            logging.error(f"El archivo posre_LIG.itp no se creó correctamente en {folder}.")
            return

        logging.info(f"Restricciones de posición para el ligando generadas correctamente en {folder}.")

        # Modificar el archivo topol.top para incluir las restricciones del ligando
        edit_topology_file(topol_file)

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al ejecutar los comandos en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado en {folder}: {e}")

# Función para editar el archivo topol.top y añadir las restricciones del ligando
def edit_topology_file(topol_file):
    try:
        # Leer el contenido del archivo topol.top
        with open(topol_file, "r") as file:
            lines = file.readlines()

        # Buscar la sección donde agregar las restricciones del ligando
        include_posres_index = None
        for i, line in enumerate(lines):
            if '#include "posre.itp"' in line:  # Buscar la línea de restricciones generales
                include_posres_index = i + 2  # Insertar justo después de esta línea

        if include_posres_index is None:
            # Si no se encuentra la sección de restricciones generales, agregarla al final
            logging.warning(f"No se encontró '#include \"posre.itp\"' en {topol_file}. Agregando restricciones al final.")
            include_posres_index = len(lines)

        # Texto a agregar al archivo topol.top para las restricciones del ligando
        ligand_posres_text = (
            "\n; Ligand position restraints\n"
            "#ifdef POSRES\n"
            '#include "posre_LIG.itp"\n'
            "#endif\n"
        )

        # Insertar el texto en el archivo topol.top
        lines.insert(include_posres_index, ligand_posres_text)

        # Guardar el archivo modificado
        with open(topol_file, "w") as file:
            file.writelines(lines)

        logging.info(f"Archivo {topol_file} modificado correctamente para incluir restricciones del ligando.")

    except Exception as e:
        logging.error(f"Error al editar {topol_file}: {e}")

# Función principal para ejecutar el proceso completo en todas las carpetas con una barra de progreso
def apply_restrictions_in_all_folders():
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'. Verifica si se han creado correctamente.")
        return

    # Usar tqdm para mostrar el progreso
    with tqdm(total=len(protein_folders), desc="Restricción del Ligando", unit="folder") as pbar:
        for folder in protein_folders:
            original_dir = os.getcwd()  # Guardar el directorio original
            try:
                # Verificar si la carpeta existe
                if not os.path.exists(folder):
                    logging.warning(f"La carpeta {folder} no existe. Saltando...")
                    pbar.update(1)
                    continue

                # Cambiar al directorio de la carpeta de la proteína
                os.chdir(folder)
                logging.info(f"Iniciando restricción del ligando en {folder}.")

                # Ejecutar el proceso de restricción del ligando
                apply_ligand_restriction(folder)

            except Exception as e:
                logging.error(f"Error en el proceso de restricción del ligando en {folder}: {e}")
            finally:
                # Volver al directorio principal, incluso si ocurre un error
                os.chdir(original_dir)
                pbar.update(1)

# Ejecutar el script
if __name__ == "__main__":
    apply_restrictions_in_all_folders()



In [ ]:
# Script automatizado para la generación de archivos de índice
# 1.0.1
#

import os
import subprocess
import logging
from tqdm import tqdm

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Función para detectar todas las carpetas que terminen en "_MDS"
def get_protein_folders():
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Función para clasificar los residuos del ligando basados en el número de átomos
def classify_ligands(gro_file):
    lig_residues = []  # Lista para almacenar residuos del ligando
    
    try:
        with open(gro_file, 'r') as file:
            atoms = []
            for line in file:
                if 'LIG' in line:  # Buscar el residuo LIG
                    residue_num = line[0:5].strip()  # Número del residuo
                    atom_name = line[10:15].strip()  # Nombre del átomo
                    atoms.append((residue_num, atom_name))

            # Contar átomos por residuo
            residue_atom_count = {}
            for residue_num, atom_name in atoms:
                if residue_num not in residue_atom_count:
                    residue_atom_count[residue_num] = 0
                residue_atom_count[residue_num] += 1

            # Clasificar ligando según el número de átomos
            for residue_num, atom_count in residue_atom_count.items():
                lig_residues.append(residue_num)

    except FileNotFoundError:
        logging.error(f"El archivo {gro_file} no se encontró.")
        return None
    except Exception as e:
        logging.error(f"Error inesperado al procesar el archivo {gro_file}: {e}")
        return None

    return lig_residues

# Función para generar el archivo index.ndx separando Ligand y combinando con la proteína
def generate_ligand_index(folder):
    try:
        # Verificar si el archivo EM.gro existe
        em_gro_file = "EM.gro"
        if not os.path.exists(em_gro_file):
            logging.warning(f"Archivo {em_gro_file} no encontrado en {folder}. Saltando...")
            return

        # Clasificar el ligando
        lig_residues = classify_ligands(em_gro_file)
        
        if not lig_residues:
            logging.error(f"No se encontraron residuos de Ligand en {em_gro_file}.")
            return

        # Convertir las listas de residuos a formato compatible con GROMACS make_ndx
        lig_residues_str = ' | '.join(lig_residues)

        # Comando gmx make_ndx para generar el archivo de índice
        make_ndx_command = f"gmx make_ndx -f {em_gro_file} -o index.ndx"
        process = subprocess.Popen(make_ndx_command, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE)

        # Crear grupos de Ligand y Protein_Ligand
        input_string = (f"r {lig_residues_str}\n"  # Crear el grupo Ligand basado en el número de residuos
                        f"name 21 Ligand\n"  # Nombra el grupo Ligand
                        f"1 | 21\n"  # Combina los grupos de proteína y ligando
                        f"name 22 Protein_Ligand\n"  # Nombra el nuevo grupo combinado
                        f"q\n")

        process.communicate(input=input_string.encode())
        process.wait()

        # Verificar si index.ndx fue creado correctamente
        if not os.path.exists("index.ndx"):
            logging.error(f"El archivo index.ndx no se creó correctamente en {folder}.")
            return

        logging.info(f"Archivo index.ndx generado correctamente en {folder}, incluyendo Ligand y Protein_Ligand.")

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al ejecutar los comandos en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado en {folder}: {e}")

# Función principal para ejecutar el proceso completo en todas las carpetas con una barra de progreso
def generate_indices_in_all_folders():
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'. Verifica si se han creado correctamente.")
        return

    # Usar tqdm para mostrar el progreso
    with tqdm(total=len(protein_folders), desc="Generación de Índices", unit="folder") as pbar:
        for folder in protein_folders:
            original_dir = os.getcwd()  # Guardar el directorio original
            try:
                # Verificar si la carpeta existe
                if not os.path.exists(folder):
                    logging.warning(f"La carpeta {folder} no existe. Saltando...")
                    pbar.update(1)
                    continue

                # Cambiar al directorio de la carpeta de la proteína
                os.chdir(folder)
                logging.info(f"Iniciando generación del archivo index.ndx en {folder}.")

                # Ejecutar el proceso de generación de índices combinando grupos
                generate_ligand_index(folder)

            except Exception as e:
                logging.error(f"Error en el proceso de generación de índices en {folder}: {e}")
            finally:
                # Volver al directorio principal, incluso si ocurre un error
                os.chdir(original_dir)
                pbar.update(1)

# Ejecutar el script
if __name__ == "__main__":
    generate_indices_in_all_folders()


In [ ]:
#NVT Equilibration
# v1.0.1
# script más robusto y fácil de mantener. El uso de pathlib mejora la legibilidad 
# y manejo de rutas, el manejo de excepciones específico proporciona información útil 
# para la depuración, y la verificación más estricta de archivos asegura que el script 
# no falle silenciosamente debido a archivos faltantes.

import os
import subprocess
import logging
import shutil

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Función para detectar todas las carpetas que terminen en "_MDS"
def get_protein_folders():
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Función para verificar si la equilibración NVT ya ha sido completada exitosamente
def is_nvt_equilibration_completed(folder):
    # Verificar si el archivo NVT.edr existe y si NVT.log contiene "Writing final coordinates"
    nvt_edr_file = os.path.join(folder, "NVT.edr")
    nvt_log_file = os.path.join(folder, "NVT.log")
    
    if not os.path.exists(nvt_edr_file):
        return False
    
    # Verificar el contenido de NVT.log para asegurar que la simulación fue exitosa
    if os.path.exists(nvt_log_file):
        with open(nvt_log_file, 'r') as log_file:
            log_content = log_file.read()
            if "Writing final coordinates" in log_content:
                logging.info(f"Equilibración NVT completada para {folder}.")
                return True
            else:
                logging.warning(f"Equilibración NVT incompleta o fallida para {folder}.")
                return False
    else:
        return False

# Función para ejecutar el proceso de equilibración NVT
def equilibrate_nvt(folder):
    try:
        # Verificar si los archivos necesarios existen
        nvt_mdp_file = "NVT.mdp"
        em_gro_file = "EM.gro"
        topol_file = "topol.top"
        index_file = "index.ndx"
        
        if not os.path.exists(nvt_mdp_file) or not os.path.exists(em_gro_file) or not os.path.exists(topol_file) or not os.path.exists(index_file):
            logging.warning(f"Archivos {nvt_mdp_file}, {em_gro_file}, {topol_file} o {index_file} faltan en {folder}. Saltando...")
            return

        # Comando gmx grompp para preparar el archivo .tpr para la simulación NVT
        grompp_command = f"gmx grompp -f {nvt_mdp_file} -c {em_gro_file} -r {em_gro_file} -p {topol_file} -n {index_file} -maxwarn 2 -o NVT.tpr"
        subprocess.run(grompp_command, shell=True, check=True)
        logging.info(f"Archivo NVT.tpr generado correctamente en {folder}.")

        # Verificar si NVT.tpr fue creado
        if not os.path.exists("NVT.tpr"):
            logging.error(f"El archivo NVT.tpr no se creó correctamente en {folder}.")
            return

        # Comando gmx mdrun para realizar la simulación NVT usando CPU-GPU Hybrid Setup
        mdrun_command = "gmx mdrun -deffnm NVT -ntmpi 1 -ntomp 20 -gpu_id 0"  # 20 hilos CPU
        subprocess.run(mdrun_command, shell=True, check=True)
        logging.info(f"Simulación NVT completada en {folder}.")

        # Verificar si NVT.edr fue creado
        if not os.path.exists("NVT.edr"):
            logging.error(f"El archivo NVT.edr no se creó correctamente en {folder}.")
            return

        # Verificar la progresión de la temperatura con gmx energy
        verify_temperature_progression(folder)

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al ejecutar los comandos en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado en {folder}: {e}")

# Función para verificar la progresión de temperatura y guardar el archivo en una carpeta de análisis
def verify_temperature_progression(folder):
    try:
        # Crear la carpeta "analisis" si no existe
        analysis_dir = os.path.join(folder, "analisis")
        if not os.path.exists(analysis_dir):
            os.makedirs(analysis_dir)
            logging.info(f"Carpeta 'analisis' creada en {folder}.")

        # Comando gmx energy para extraer la temperatura y guardar en temperature.xvg
        energy_command = f"echo 17 0 | gmx energy -f NVT.edr -o temperature.xvg"
        subprocess.run(energy_command, shell=True, check=True)
        logging.info(f"Archivo temperature.xvg generado correctamente en {folder}.")

        # Verificar si temperature.xvg fue creado
        if not os.path.exists("temperature.xvg"):
            logging.error(f"El archivo temperature.xvg no se creó correctamente en {folder}.")
            return

        # Mover el archivo temperature.xvg a la carpeta "analisis"
        shutil.move("temperature.xvg", os.path.join(analysis_dir, "temperature.xvg"))
        logging.info(f"Archivo temperature.xvg movido a la carpeta 'analisis' en {folder}.")

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al verificar la progresión de temperatura en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado en {folder}: {e}")

# Función principal para ejecutar el proceso completo en todas las carpetas
def run_nvt_in_all_folders():
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'. Verifica si se han creado correctamente.")
        return

    for folder in protein_folders:
        original_dir = os.getcwd()  # Guardar el directorio original
        try:
            # Verificar si la equilibración NVT ya ha sido completada exitosamente
            if is_nvt_equilibration_completed(folder):
                continue  # Saltar esta carpeta si la simulación ya fue completada exitosamente

            # Cambiar al directorio de la carpeta de la proteína
            os.chdir(folder)
            logging.info(f"Iniciando equilibración NVT en {folder}.")

            # Ejecutar el proceso de equilibración NVT
            equilibrate_nvt(folder)

        except Exception as e:
            logging.error(f"Error en el proceso de equilibración NVT en {folder}: {e}")
        finally:
            # Volver al directorio principal, incluso si ocurre un error
            os.chdir(original_dir)

# Ejecutar el script
if __name__ == "__main__":
    run_nvt_in_all_folders()

In [ ]:
# NPT Equilibration

import os
import subprocess
import logging
import shutil

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Función para detectar todas las carpetas que terminen en "_MDS"
def get_protein_folders():
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Función para verificar si la equilibración NPT ya ha sido completada exitosamente
def is_npt_equilibration_completed(folder):
    # Verificar si el archivo NPT.edr existe y si NPT.log contiene "Writing final coordinates"
    npt_edr_file = os.path.join(folder, "NPT.edr")
    npt_log_file = os.path.join(folder, "NPT.log")
    
    if not os.path.exists(npt_edr_file):
        return False
    
    # Verificar el contenido de NPT.log para asegurar que la simulación fue exitosa
    if os.path.exists(npt_log_file):
        with open(npt_log_file, 'r') as log_file:
            log_content = log_file.read()
            if "Writing final coordinates" in log_content:
                logging.info(f"Equilibración NPT completada para {folder}.")
                return True
            else:
                logging.warning(f"Equilibración NPT incompleta o fallida para {folder}.")
                return False
    else:
        return False

# Función para ejecutar el proceso de equilibración NPT
def equilibrate_npt(folder):
    try:
        # Verificar si los archivos necesarios existen
        npt_mdp_file = "NPT.mdp"
        nvt_gro_file = "NVT.gro"
        topol_file = "topol.top"
        index_file = "index.ndx"
        
        if not os.path.exists(npt_mdp_file) or not os.path.exists(nvt_gro_file) or not os.path.exists(topol_file) or not os.path.exists(index_file):
            logging.warning(f"Archivos {npt_mdp_file}, {nvt_gro_file}, {topol_file} o {index_file} faltan en {folder}. Saltando...")
            return

        # Comando gmx grompp para preparar el archivo .tpr para la simulación NPT
        grompp_command = f"gmx grompp -f {npt_mdp_file} -c {nvt_gro_file} -r {nvt_gro_file} -p {topol_file} -n {index_file} -maxwarn 2 -o NPT.tpr"
        subprocess.run(grompp_command, shell=True, check=True)
        logging.info(f"Archivo NPT.tpr generado correctamente en {folder}.")

        # Verificar si NPT.tpr fue creado
        if not os.path.exists("NPT.tpr"):
            logging.error(f"El archivo NPT.tpr no se creó correctamente en {folder}.")
            return

        # Comando gmx mdrun para realizar la simulación NPT usando GPU
        mdrun_command = "gmx mdrun -deffnm NPT -gpu_id 0 -nb gpu -pme gpu"  # Usar la GPU 0 para interacciones no enlazadas y PME
        subprocess.run(mdrun_command, shell=True, check=True)
        logging.info(f"Simulación NPT completada en {folder}.")

        # Verificar si NPT.edr fue creado
        if not os.path.exists("NPT.edr"):
            logging.error(f"El archivo NPT.edr no se creó correctamente en {folder}.")
            return

        # Verificar la progresión de la presión y la densidad
        verify_pressure_and_density(folder)

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al ejecutar los comandos en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado en {folder}: {e}")

# Función para verificar la progresión de presión y densidad y guardar los archivos en una carpeta de análisis
def verify_pressure_and_density(folder):
    try:
        # Crear la carpeta "analisis" si no existe
        analysis_dir = os.path.join(folder, "analisis")
        if not os.path.exists(analysis_dir):
            os.makedirs(analysis_dir)
            logging.info(f"Carpeta 'analisis' creada en {folder}.")

        # Comando gmx energy para extraer la presión y guardar en pressure.xvg
        pressure_command = f"echo 19 0 | gmx energy -f NPT.edr -o pressure.xvg"
        subprocess.run(pressure_command, shell=True, check=True)
        logging.info(f"Archivo pressure.xvg generado correctamente en {folder}.")

        # Verificar si pressure.xvg fue creado
        if not os.path.exists("pressure.xvg"):
            logging.error(f"El archivo pressure.xvg no se creó correctamente en {folder}.")
            return

        # Mover el archivo pressure.xvg a la carpeta "analisis"
        shutil.move("pressure.xvg", os.path.join(analysis_dir, "pressure.xvg"))
        logging.info(f"Archivo pressure.xvg movido a la carpeta 'analisis' en {folder}.")

        # Comando gmx energy para extraer la densidad y guardar en density.xvg
        density_command = f"echo 25 0 | gmx energy -f NPT.edr -o density.xvg"
        subprocess.run(density_command, shell=True, check=True)
        logging.info(f"Archivo density.xvg generado correctamente en {folder}.")

        # Verificar si density.xvg fue creado
        if not os.path.exists("density.xvg"):
            logging.error(f"El archivo density.xvg no se creó correctamente en {folder}.")
            return

        # Mover el archivo density.xvg a la carpeta "analisis"
        shutil.move("density.xvg", os.path.join(analysis_dir, "density.xvg"))
        logging.info(f"Archivo density.xvg movido a la carpeta 'analisis' en {folder}.")

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al verificar la progresión de presión y densidad en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado en {folder}: {e}")

# Función principal para ejecutar el proceso completo en todas las carpetas
def run_npt_in_all_folders():
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'. Verifica si se han creado correctamente.")
        return

    for folder in protein_folders:
        original_dir = os.getcwd()  # Guardar el directorio original
        try:
            # Verificar si la equilibración NPT ya ha sido completada exitosamente
            if is_npt_equilibration_completed(folder):
                continue  # Saltar esta carpeta si la simulación ya fue completada exitosamente

            # Cambiar al directorio de la carpeta de la proteína
            os.chdir(folder)
            logging.info(f"Iniciando equilibración NPT en {folder}.")

            # Ejecutar el proceso de equilibración NPT
            equilibrate_npt(folder)

        except Exception as e:
            logging.error(f"Error en el proceso de equilibración NPT en {folder}: {e}")
        finally:
            # Volver al directorio principal, incluso si ocurre un error
            os.chdir(original_dir)

# Ejecutar el script
if __name__ == "__main__":
    run_npt_in_all_folders()


In [ ]:
#Molecular dinamics Simulation
# 1.0.1
# Serial (Automatically adjusts to the number of available CPU cores and GPUs in your system)

import os
import subprocess
import logging
import shutil

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Función para detectar todas las carpetas que terminen en "_MDS"
def get_protein_folders():
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Función para verificar si la simulación MD ya se completó
def is_md_simulation_completed(folder):
    md_log_file = os.path.join(folder, "analisis", "MD.log")
    if os.path.exists(md_log_file):
        with open(md_log_file, 'r') as file:
            for line in file:
                if "Finished mdrun on rank 0" in line:
                    logging.info(f"Simulación MD ya completada en {folder}. Saltando...")
                    return True
    return False

# Función para ejecutar la simulación de dinámica molecular
def run_md_simulation(folder):
    try:
        # Verificar si la simulación ya fue completada
        if is_md_simulation_completed(folder):
            return

        # Verificar si los archivos necesarios existen
        md_mdp_file = "MD.mdp"
        npt_gro_file = "NPT.gro"
        npt_cpt_file = "NPT.cpt"
        topol_file = "topol.top"
        index_file = "index.ndx"
        
        if not os.path.exists(md_mdp_file) or not os.path.exists(npt_gro_file) or not os.path.exists(npt_cpt_file) or not os.path.exists(topol_file) or not os.path.exists(index_file):
            logging.warning(f"Archivos {md_mdp_file}, {npt_gro_file}, {npt_cpt_file}, {topol_file} o {index_file} faltan en {folder}. Saltando...")
            return

        # Comando gmx grompp para preparar el archivo .tpr para la simulación MD
        grompp_command = f"gmx grompp -f {md_mdp_file} -c {npt_gro_file} -t {npt_cpt_file} -p {topol_file} -n {index_file} -maxwarn 2 -o MD.tpr"
        subprocess.run(grompp_command, shell=True, check=True)
        logging.info(f"Archivo MD.tpr generado correctamente en {folder}.")

        # Verificar si MD.tpr fue creado
        if not os.path.exists("MD.tpr"):
            logging.error(f"El archivo MD.tpr no se creó correctamente en {folder}.")
            return

        # Comando gmx mdrun para realizar la simulación MD utilizando GPU
        mdrun_command = "gmx mdrun -deffnm MD -gpu_id 0 -nb gpu -pme gpu"  # Usar la GPU 0 para interacciones no enlazadas y PME
        subprocess.run(mdrun_command, shell=True, check=True)
        logging.info(f"Simulación MD completada en {folder}.")

        # Verificar y organizar los resultados de la simulación
        verify_md_simulation(folder)

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al ejecutar los comandos en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado en {folder}: {e}")

# Función para verificar y organizar los resultados de la simulación MD
def verify_md_simulation(folder):
    try:
        # Crear la carpeta "analisis" si no existe
        analysis_dir = os.path.join(folder, "analisis")
        if not os.path.exists(analysis_dir):
            os.makedirs(analysis_dir)
            logging.info(f"Carpeta 'analisis' creada en {folder}.")

        # Mover los archivos relevantes de la simulación MD a la carpeta de análisis
        for file in ["MD.log", "MD.xtc", "MD.edr", "MD.tpr", "MD.gro"]:
            if os.path.exists(file):
                shutil.move(file, os.path.join(analysis_dir, file))
                logging.info(f"Archivo {file} movido a la carpeta 'analisis' en {folder}.")
            else:
                logging.warning(f"Archivo {file} no encontrado en {folder}. No se pudo mover.")

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al verificar y organizar los resultados en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado en {folder}: {e}")

# Función principal para ejecutar el proceso completo en todas las carpetas
def run_md_in_all_folders():
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'. Verifica si se han creado correctamente.")
        return

    for folder in protein_folders:
        original_dir = os.getcwd()  # Guardar el directorio original
        try:
            # Verificar si la carpeta existe
            if not os.path.exists(folder):
                logging.warning(f"La carpeta {folder} no existe. Saltando...")
                continue

            # Cambiar al directorio de la carpeta de la proteína
            os.chdir(folder)
            logging.info(f"Iniciando simulación MD en {folder}.")

            # Ejecutar el proceso de simulación MD
            run_md_simulation(folder)

        except Exception as e:
            logging.error(f"Error en el proceso de simulación MD en {folder}: {e}")
        finally:
            # Volver al directorio principal, incluso si ocurre un error
            os.chdir(original_dir)

# Ejecutar el script
if __name__ == "__main__":
    run_md_in_all_folders()



In [ ]:
import os
import logging

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Función para detectar todas las carpetas que terminen en "_MDS"
def get_protein_folders():
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Función para verificar la presencia de los archivos necesarios
def check_files_in_analysis(folder):
    analysis_dir = os.path.join(folder, "analisis")

    # Verificar si la carpeta de análisis existe
    if not os.path.exists(analysis_dir):
        logging.warning(f"La carpeta 'analisis' no existe en {folder}.")
        return

    # Listar archivos presentes en la carpeta "analisis"
    logging.info(f"Archivos encontrados en {analysis_dir}: {os.listdir(analysis_dir)}")

    # Definir las rutas de los archivos que se deben verificar
    required_files = ["MD.tpr", "MD.xtc", "MD.edr"]
    missing_files = []

    # Verificar la existencia de cada archivo
    for file_name in required_files:
        file_path = os.path.join(analysis_dir, file_name)
        if not os.path.exists(file_path) or '#' in file_name:
            missing_files.append(file_name)

    if missing_files:
        logging.warning(f"Faltan los siguientes archivos en {analysis_dir}: {', '.join(missing_files)}")
    else:
        logging.info(f"Todos los archivos necesarios están presentes en {analysis_dir}.")

# Función principal para revisar todas las carpetas de proteínas
def check_all_folders():
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'.")
        return

    for folder in protein_folders:
        logging.info(f"Revisando archivos en {folder}.")
        check_files_in_analysis(folder)

# Ejecutar el script
if __name__ == "__main__":
    check_all_folders()


In [ ]:
#Individual analisis
#Parallel (automatically adjusts to the number of CPU cores in your system)

import os
import subprocess
import logging
import shutil

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Guardar el directorio original
original_dir = os.getcwd()

# Función para detectar todas las carpetas que terminen en "_MDS"
def get_protein_folders():
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Función para verificar si los archivos necesarios están presentes
def are_files_present(folder):
    required_files = ["MD.tpr", "MD.xtc", "MD.edr"]
    analysis_dir = os.path.join(folder, "analisis")
    if not os.path.exists(analysis_dir):
        logging.warning(f"La carpeta 'analisis' no existe en {folder}.")
        return False

    files_in_folder = os.listdir(analysis_dir)
    missing_files = [f for f in required_files if f not in files_in_folder]

    if missing_files:
        logging.warning(f"Faltan los siguientes archivos en {folder}/analisis: {', '.join(missing_files)}")
        return False
    
    logging.info(f"Todos los archivos necesarios están presentes en {folder}/analisis.")
    return True

# Función para ejecutar el análisis de MDS en una carpeta
def run_mds_analysis(folder):
    try:
        analysis_dir = os.path.join(folder, "analisis")
        os.chdir(analysis_dir)

        # Definir la ruta de los archivos necesarios
        md_tpr_file = os.path.join(analysis_dir, "MD.tpr")
        md_xtc_file = os.path.join(analysis_dir, "MD.xtc")
        md_edr_file = os.path.join(analysis_dir, "MD.edr")

        # Comandos para el análisis
        recenter_command = f"echo 1 0 | gmx trjconv -s {md_tpr_file} -f {md_xtc_file} -o MD_center.xtc -center -pbc mol -ur compact"
        subprocess.run(recenter_command, shell=True, check=True)
        logging.info(f"Coordenadas recentradas y guardadas en MD_center.xtc en {folder}.")

        extract_frame_command = f"gmx trjconv -s {md_tpr_file} -f MD_center.xtc -o start.pdb -dump 0"
        subprocess.run(extract_frame_command, shell=True, check=True)
        logging.info(f"Frame inicial extraído y guardado en start.pdb en {folder}.")

        # Calcular RMSD del ligando respecto a la proteína
        rmsd_lig_prot_command = f"echo 13 1 | gmx rms -s {md_tpr_file} -f MD_center.xtc -tu ns -o rmsd_lig_prot.xvg"
        subprocess.run(rmsd_lig_prot_command, shell=True, check=True)
        logging.info(f"RMSD del ligando calculado y guardado en rmsd_lig_prot.xvg en {folder}.")

        # Calcular RMSD del backbone
        rmsd_backbone_command = f"echo 4 4 | gmx rms -s {md_tpr_file} -f MD_center.xtc -tu ns -o rmsd_backbone.xvg"
        subprocess.run(rmsd_backbone_command, shell=True, check=True)
        logging.info(f"RMSD del backbone calculado y guardado en rmsd_backbone.xvg en {folder}.")

        # Calcular otros análisis como RMSF, enlaces de hidrógeno, radio de giro, energía, etc.
        rmsf_command = f"echo 4 | gmx rmsf -s {md_tpr_file} -f MD_center.xtc -o rmsf.xvg"
        subprocess.run(rmsf_command, shell=True, check=True)
        logging.info(f"RMSF calculado y guardado en rmsf.xvg en {folder}.")

        hbond_command = f"echo 1 13 | gmx hbond -s {md_tpr_file} -f MD_center.xtc -num hb.xvg -tu ns"
        subprocess.run(hbond_command, shell=True, check=True)
        logging.info(f"Enlaces de hidrógeno calculados y guardados en hb.xvg en {folder}.")

        gyrate_command = f"echo 1 | gmx gyrate -s {md_tpr_file} -f MD_center.xtc -o gyrate1.xvg"
        subprocess.run(gyrate_command, shell=True, check=True)
        logging.info(f"Radio de giro calculado y guardado en gyrate1.xvg en {folder}.")

        energy_command = f"gmx energy -f {md_edr_file} -o energy1.xvg"
        subprocess.run(energy_command, shell=True, check=True)
        logging.info(f"Energía calculada y guardada en energy1.xvg en {folder}.")

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al ejecutar los comandos en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado en {folder}: {e}")
    finally:
        os.chdir(original_dir)

# Función principal para ejecutar el análisis solo en las carpetas con simulaciones completas
def run_analysis_for_completed_simulations():
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'. Verifica si se han creado correctamente.")
        return

    for folder in protein_folders:
        logging.info(f"Revisando archivos en {folder}.")
        
        if are_files_present(folder):
            logging.info(f"Iniciando análisis de MDS en {folder}.")
            run_mds_analysis(folder)
        else:
            logging.warning(f"Saltando análisis para {folder} ya que no tiene todos los archivos necesarios.")

# Ejecutar el script
if __name__ == "__main__":
    run_analysis_for_completed_simulations()


In [ ]:
import os
import logging

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Cambiar al directorio correcto (ajusta la ruta si es necesario)
root_dir = "/media/cruz/TOSHIBA EXT/CENO_PL"
os.chdir(root_dir)
print(f"Directorio actual: {os.getcwd()}")

# Función para detectar todas las carpetas que terminen en "_MDS"
def get_protein_folders():
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Función principal para ejecutar el análisis solo en las carpetas con simulaciones completas
def run_analysis_for_completed_simulations():
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'. Verifica si se han creado correctamente.")
        return

    for folder in protein_folders:
        logging.info(f"Revisando archivos en {folder}.")
        # Aquí continuaría el análisis específico para cada carpeta

# Ejecutar el script
if __name__ == "__main__":
    run_analysis_for_completed_simulations()


In [ ]:
import os

print(f"Directorio actual: {os.getcwd()}")


In [ ]:
# Global analisys

import os
import logging
import matplotlib.pyplot as plt

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Directory to save comparison results (assuming current directory)
comparison_dir = os.path.join(os.getcwd(), "analysis_comparison")
if not os.path.exists(comparison_dir):
    os.makedirs(comparison_dir)

# Function to detect all folders that end with '_MDS'
def get_protein_folders():
    """Detect all folders in the current directory that end with '_MDS'."""
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Function to check if the analysis files exist
def check_files_exist(folder, analysis_file):
    """Check if the required analysis file exists in the given folder."""
    data_path = os.path.join(folder, "analisis", analysis_file)
    logging.debug(f"Trying to read file: {data_path}")
    return os.path.exists(data_path)

# Function to extract data from .xvg files
def extract_analysis_data(folder, analysis_file, column_index=1):
    """Helper function to extract data from an .xvg file, choosing a specific column."""
    data_path = os.path.join(folder, "analisis", analysis_file)
    if os.path.exists(data_path):
        times, values = [], []
        with open(data_path, 'r') as f:
            for line in f:
                if not line.startswith(('@', '#')) and line.strip():  # Ignore comments and blank lines
                    parts = line.split()
                    if len(parts) > column_index:
                        try:
                            time = float(parts[0])
                            value = float(parts[column_index])
                            times.append(time)
                            values.append(value)
                        except ValueError:
                            logging.warning(f"Error converting values in line: {line.strip()}")
                    else:
                        logging.warning(f"Line in {analysis_file} does not have enough columns: {line.strip()}")
        if times and values:
            return times, values
        else:
            logging.warning(f"No valid data found in {analysis_file}")
            return None, None
    else:
        logging.info(f"{analysis_file} not found in {data_path}. Skipping folder.")
        return None, None

# Function to save data to a file
def save_data_to_file(data_dict, analysis_type, ylabel):
    """Save the data used for plotting to a .txt file."""
    if not data_dict:  # Skip if no data is available
        logging.info(f"No data to save for {analysis_type}")
        return
    
    data_file = os.path.join(comparison_dir, f"{analysis_type}_data.txt")
    with open(data_file, 'w') as f:
        f.write(f"# {analysis_type} - {ylabel}\n")
        for protein, (times, values) in data_dict.items():
            f.write(f"\n# Data for {protein}\n")
            f.write("Time(ns)\tValue\n")
            for t, v in zip(times, values):
                f.write(f"{t}\t{v}\n")
    logging.info(f"Data saved in {data_file}")

# Function to generate comparison plots
def plot_comparison(data_dict, ylabel, title, output_file):
    """Generate comparison plots."""
    if not data_dict:  # Skip if no data is available
        logging.info(f"No data to plot for {title}.")
        return
    
    plt.figure(figsize=(10, 6))
    for protein, (times, values) in data_dict.items():
        if times and values:
            plt.plot(times, values, label=protein)
    
    plt.xlabel("Time (ns)")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.grid(True)
    
    output_path = os.path.join(comparison_dir, output_file)
    plt.savefig(output_path)
    plt.close()
    logging.info(f"Comparison plot saved: {output_file}")

# Function to run all comparisons
def run_mds_comparison():
    # Detect all protein folders ending in '_MDS'
    protein_folders = get_protein_folders()
    if not protein_folders:
        logging.warning("No protein folders ending with '_MDS' found in the current directory.")
        return

    # Analysis types and corresponding file names from the first script
    analysis_types = {
    'RMSD_Ligand_Protein': {'file': 'rmsd_ligand.xvg', 'ylabel': 'RMSD Ligand-Protein (nm)', 'title': 'RMSD Ligand-Protein Comparison', 'output': 'rmsd_lig_prot_comparison.png', 'column': 1},
    'RMSD_Backbone_Protein': {'file': 'rmsd_backbone.xvg', 'ylabel': 'RMSD Backbone-Protein (nm)', 'title': 'RMSD Backbone-Protein Comparison', 'output': 'rmsd_backbone_comparison.png', 'column': 1},
    'RMSF': {'file': 'rmsf.xvg', 'ylabel': 'RMSF (nm)', 'title': 'RMSF Comparison', 'output': 'rmsf_comparison.png', 'column': 1},
    'Hydrogen Bonds': {'file': 'hb.xvg', 'ylabel': 'Number of Hydrogen Bonds', 'title': 'Hydrogen Bond Comparison', 'output': 'hbond_comparison.png', 'column': 1},
    'Gyration Radius': {'file': 'gyrate1.xvg', 'ylabel': 'Gyration Radius (nm)', 'title': 'Gyration Radius Comparison', 'output': 'gyration_comparison.png', 'column': 1},
    'Energy': {'file': 'energy1.xvg', 'ylabel': 'Energy (kJ/mol)', 'title': 'Energy Comparison', 'output': 'energy_comparison.png', 'column': 1}
}

    
    for analysis, details in analysis_types.items():
        data_dict = {}
        for folder in protein_folders:
            protein_name = os.path.basename(folder)
            # Check if the required file exists before processing
            if check_files_exist(folder, details['file']):
                times, values = extract_analysis_data(folder, details['file'], details['column'])
                if times and values:
                    data_dict[protein_name] = (times, values)
            else:
                logging.info(f"Exiting {folder} for {details['title']} analysis due to missing file.")
        
        if data_dict:  # Only generate the plot if there is valid data
            plot_comparison(data_dict, details['ylabel'], details['title'], details['output'])
            # Save the data used for the plot to a file
            save_data_to_file(data_dict, analysis, details['ylabel'])
        else:
            logging.info(f"No valid data found for {details['title']}. No plot generated.")

# Run the comparison script
if __name__ == "__main__":
    run_mds_comparison()


In [ ]:




# Alto! partir de aqui los siquientes modulos estan en desarrollo o revision...

print("A partir de aqui los siquientes modulos estan en desarrollo o revision!")





In [ ]:
#Individual analysis
#Parallel

import os
import subprocess
import logging
import concurrent.futures

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Función para detectar todas las carpetas que terminen en "_MDS"
def get_protein_folders():
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Función para ejecutar el proceso de análisis de MDS
def run_mds_analysis(folder):
    try:
        # Definir la ruta donde están los archivos de la simulación
        analysis_dir = os.path.join(folder, "analisis")

        # Verificar si los archivos necesarios existen dentro de la carpeta "analisis"
        md_tpr_file = os.path.join(analysis_dir, "MD.tpr")
        md_xtc_file = os.path.join(analysis_dir, "MD.xtc")
        md_edr_file = os.path.join(analysis_dir, "MD.edr")
        index_file = os.path.join(folder, "index.ndx")  # El archivo index.ndx está en la carpeta principal de la proteína

        # Verificar si todos los archivos necesarios están presentes
        required_files = [md_tpr_file, md_xtc_file, md_edr_file, index_file]
        if not all([os.path.exists(file) for file in required_files]):
            logging.error(f"Faltan archivos necesarios en {folder}. Saltando análisis.")
            return

        # Crear la carpeta "analisis" si no existe
        if not os.path.exists(analysis_dir):
            os.makedirs(analysis_dir)
            logging.info(f"Carpeta 'analisis' creada en {folder}.")

        # Recentrar y reajustar coordenadas
        recenter_command = f"echo 1 0 | gmx trjconv -s {md_tpr_file} -f {md_xtc_file} -o MD_center.xtc -center -pbc mol -ur compact"
        subprocess.run(recenter_command, shell=True, check=True)
        logging.info(f"Coordenadas recentradas y guardadas en MD_center.xtc en {folder}.")
        
        # Verificar si el archivo MD_center.xtc fue creado
        md_center_xtc = os.path.join(analysis_dir, "MD_center.xtc")
        if not os.path.exists(md_center_xtc):
            logging.error(f"El archivo MD_center.xtc no se creó correctamente en {folder}.")
            return

        # Extraer el primer frame de la trayectoria (t = 0 ns)
        extract_frame_command = f"gmx trjconv -s {md_tpr_file} -f {md_center_xtc} -o start.pdb -dump 0"
        subprocess.run(extract_frame_command, shell=True, check=True)
        logging.info(f"Frame inicial extraído y guardado en start.pdb en {folder}.")
        
        # Verificar si el archivo start.pdb fue creado
        start_pdb = os.path.join(analysis_dir, "start.pdb")
        if not os.path.exists(start_pdb):
            logging.error(f"El archivo start.pdb no se creó correctamente en {folder}.")
            return

        # Calcular RMSD del backbone
        rmsd_backbone_command = f"echo 4 4 | gmx rms -s {md_tpr_file} -f {md_center_xtc} -tu ns -o rmsd_backbone.xvg"
        subprocess.run(rmsd_backbone_command, shell=True, check=True)
        logging.info(f"RMSD del backbone calculado y guardado en rmsd_backbone.xvg en {folder}.")

        rmsd_backbone_xvg = os.path.join(analysis_dir, "rmsd_backbone.xvg")
        if not os.path.exists(rmsd_backbone_xvg):
            logging.error(f"El archivo rmsd_backbone.xvg no se creó correctamente en {folder}.")
            return

        # Calcular RMSD del ligando respecto a la proteína
        rmsd_ligand_command = f"echo 1 13 | gmx rms -s {md_tpr_file} -f {md_center_xtc} -tu ns -o rmsd_ligand.xvg"
        subprocess.run(rmsd_ligand_command, shell=True, check=True)
        logging.info(f"RMSD del ligando respecto a la proteína calculado y guardado en rmsd_ligand.xvg en {folder}.")

        rmsd_ligand_xvg = os.path.join(analysis_dir, "rmsd_ligand.xvg")
        if not os.path.exists(rmsd_ligand_xvg):
            logging.error(f"El archivo rmsd_ligand.xvg no se creó correctamente en {folder}.")
            return

        # Calcular RMSF
        rmsf_command = f"echo 4 | gmx rmsf -s {md_tpr_file} -f {md_center_xtc} -o rmsf.xvg"
        subprocess.run(rmsf_command, shell=True, check=True)
        logging.info(f"RMSF calculado y guardado en rmsf.xvg en {folder}.")
        
        rmsf_xvg = os.path.join(analysis_dir, "rmsf.xvg")
        if not os.path.exists(rmsf_xvg):
            logging.error(f"El archivo rmsf.xvg no se creó correctamente en {folder}.")
            return

        # Calcular enlaces de hidrógeno
        hbond_command = f"echo 1 13 | gmx hbond -s {md_tpr_file} -f {md_center_xtc} -num hb.xvg -tu ns"
        subprocess.run(hbond_command, shell=True, check=True)
        logging.info(f"Enlaces de hidrógeno calculados y guardados en hb.xvg en {folder}.")
        
        hb_xvg = os.path.join(analysis_dir, "hb.xvg")
        if not os.path.exists(hb_xvg):
            logging.error(f"El archivo hb.xvg no se creó correctamente en {folder}.")
            return

        # Calcular radio de giro
        gyrate_command = f"echo 1 | gmx gyrate -s {md_tpr_file} -f {md_center_xtc} -o gyrate1.xvg"
        subprocess.run(gyrate_command, shell=True, check=True)
        logging.info(f"Radio de giro calculado y guardado en gyrate1.xvg en {folder}.")
        
        gyrate_xvg = os.path.join(analysis_dir, "gyrate1.xvg")
        if not os.path.exists(gyrate_xvg):
            logging.error(f"El archivo gyrate1.xvg no se creó correctamente en {folder}.")
            return

        # Calcular energía
        energy_command = f"gmx energy -f {md_edr_file} -o energy1.xvg"
        subprocess.run(energy_command, shell=True, check=True)
        logging.info(f"Energía calculada y guardada en energy1.xvg en {folder}.")
        
        energy_xvg = os.path.join(analysis_dir, "energy1.xvg")
        if not os.path.exists(energy_xvg):
            logging.error(f"El archivo energy1.xvg no se creó correctamente en {folder}.")
            return

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al ejecutar los comandos en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado en {folder}: {e}")

# Función principal para ejecutar el proceso en todas las carpetas de forma paralela
def run_mds_analysis_in_all_folders():
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'. Verifica si se han creado correctamente.")
        return

    # Usar concurrent.futures para paralelizar el análisis con ProcessPoolExecutor
    with concurrent.futures.ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
        executor.map(run_mds_analysis, protein_folders)

# Ejecutar el script
if __name__ == "__main__":
    run_mds_analysis_in_all_folders()


In [ ]:
#Molecular dinamics simulation 
#Problema con duplicacion de carpetas corregido
#Ahora detecta que proteinas ya fueron simuladas con exito y evita volverlas a simular en caso de que el script se corra de nuevo.

import os
import subprocess
import logging

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Función para detectar todas las carpetas que terminen en "_MDS"
def get_protein_folders():
    return [folder for folder in os.listdir() if os.path.isdir(folder) and folder.endswith("_MDS")]

# Función para verificar si la simulación ya ha sido ejecutada exitosamente
def is_simulation_completed(folder):
    # Verifica si el archivo MD.log existe en el directorio principal de la carpeta de la proteína
    md_log_file = os.path.join(folder, "MD.log")
    
    # Si el archivo no existe, la simulación no ha sido completada
    if not os.path.exists(md_log_file):
        return False
    
    # Leer el archivo MD.log para verificar si la simulación terminó correctamente
    try:
        with open(md_log_file, 'r') as log_file:
            log_content = log_file.read()
            # Verificar si contiene la frase "Writing final coordinates"
            if "Writing final coordinates" in log_content:
                logging.info(f"Simulación exitosa para {folder}.")
                return True
            else:
                logging.warning(f"Simulación incompleta o fallida para {folder}.")
                return False
    except Exception as e:
        logging.error(f"Error al leer el archivo MD.log en {folder}: {e}")
        return False

# Función para ejecutar la simulación de dinámica molecular
def run_md_simulation(folder):
    try:
        # Verificar si los archivos necesarios existen
        md_mdp_file = os.path.join(folder, "MD.mdp")
        npt_gro_file = os.path.join(folder, "NPT.gro")
        npt_cpt_file = os.path.join(folder, "NPT.cpt")
        topol_file = os.path.join(folder, "topol.top")
        index_file = os.path.join(folder, "index.ndx")
        
        if not os.path.exists(md_mdp_file) or not os.path.exists(npt_gro_file) or not os.path.exists(npt_cpt_file) or not os.path.exists(topol_file) or not os.path.exists(index_file):
            logging.warning(f"Archivos {md_mdp_file}, {npt_gro_file}, {npt_cpt_file}, {topol_file} o {index_file} faltan en {folder}. Saltando...")
            return

        # Comando gmx grompp para preparar el archivo .tpr para la simulación MD
        grompp_command = f"gmx grompp -f {md_mdp_file} -c {npt_gro_file} -t {npt_cpt_file} -p {topol_file} -n {index_file} -maxwarn 2 -o {os.path.join(folder, 'MD.tpr')}"
        subprocess.run(grompp_command, shell=True, check=True)
        logging.info(f"Archivo MD.tpr generado correctamente en {folder}.")

        # Verificar si MD.tpr fue creado
        if not os.path.exists(os.path.join(folder, "MD.tpr")):
            logging.error(f"El archivo MD.tpr no se creó correctamente en {folder}.")
            return

        # Comando gmx mdrun para realizar la simulación MD utilizando GPU
        mdrun_command = f"gmx mdrun -deffnm {os.path.join(folder, 'MD')} -gpu_id 0 -nb gpu -pme gpu"
        subprocess.run(mdrun_command, shell=True, check=True)
        logging.info(f"Simulación MD completada en {folder}.")

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al ejecutar los comandos en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado en {folder}: {e}")

# Función principal para ejecutar el proceso completo en todas las carpetas
def run_md_in_all_folders():
    protein_folders = get_protein_folders()

    if not protein_folders:
        logging.warning("No se encontraron carpetas de proteínas con el sufijo '_MDS'. Verifica si se han creado correctamente.")
        return

    for folder in protein_folders:
        try:
            # Verificar si la simulación ya ha sido completada exitosamente
            if is_simulation_completed(folder):
                continue  # Saltar esta carpeta si la simulación ya fue completada exitosamente

            logging.info(f"Iniciando simulación MD en {folder}.")
            # Ejecutar el proceso de simulación MD
            run_md_simulation(folder)

        except Exception as e:
            logging.error(f"Error en el proceso de simulación MD en {folder}: {e}")

# Ejecutar el script
if __name__ == "__main__":
    run_md_in_all_folders()


In [ ]:
# Analisis    individual, con listado de carpetas



import os
import subprocess
import logging
import shutil

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(asctime)s - %(message)s')

# Base path de las carpetas de proteínas
base_path = "/home/cruz/Escritorio/MDS_Manual_3/"

# Lista de carpetas de proteínas generadas anteriormente
protein_folders = [
    "E5_15_P-C-L_output", "E5_21_P-C-L_output", "E5_25_P-C-L_output", "E5_54_P-C-L_output",
    "E5_16_P-C-L_output", "E5_22_P-C-L_output", "E5_30_P-C-L_output", "E5_55_P-C-L_output",
    "E5_1_P-C-L_output",  "E5_24_P-C-L_output", "E5_47_P-C-L_output", "E5_8_P-C-L_output"
]

# Función para ejecutar el proceso de análisis de MDS
def run_mds_analysis(folder):
    try:
        # Definir la ruta donde están los archivos de la simulación usando rutas absolutas
        analysis_dir = os.path.join(base_path, folder, "analisis")
        
        # Verificar si los archivos necesarios existen dentro de la carpeta "analisis"
        md_tpr_file = os.path.join(analysis_dir, "MD.tpr")
        md_xtc_file = os.path.join(analysis_dir, "MD.xtc")
        md_edr_file = os.path.join(analysis_dir, "MD.edr")
        index_file = os.path.join(base_path, folder, "index.ndx")  # El archivo index.ndx está en la carpeta principal de la proteína

        # Imprimir las rutas de los archivos que se están buscando
        print(f"Buscando archivo TPR en: {md_tpr_file}")
        print(f"Buscando archivo XTC en: {md_xtc_file}")
        print(f"Buscando archivo EDR en: {md_edr_file}")
        print(f"Buscando archivo INDEX en: {index_file}")

        # Verificar la existencia de cada archivo
        if not os.path.exists(md_tpr_file):
            logging.warning(f"Archivo {md_tpr_file} no encontrado.")
        if not os.path.exists(md_xtc_file):
            logging.warning(f"Archivo {md_xtc_file} no encontrado.")
        if not os.path.exists(md_edr_file):
            logging.warning(f"Archivo {md_edr_file} no encontrado.")
        if not os.path.exists(index_file):
            logging.warning(f"Archivo {index_file} no encontrado.")

        # Si alguno de los archivos no existe, saltar la carpeta
        if not (os.path.exists(md_tpr_file) and os.path.exists(md_xtc_file) and os.path.exists(md_edr_file) and os.path.exists(index_file)):
            logging.warning(f"Archivos necesarios faltan en {folder}. Saltando...")
            return

        # Recentrar y reajustar coordenadas
        recenter_command = f"echo 1 0 | gmx trjconv -s {md_tpr_file} -f {md_xtc_file} -o MD_center.xtc -center -pbc mol -ur compact"
        subprocess.run(recenter_command, shell=True, check=True)
        logging.info(f"Coordenadas recentradas y guardadas en MD_center.xtc en {folder}.")
        shutil.move("MD_center.xtc", os.path.join(analysis_dir, "MD_center.xtc"))

        # Extraer el primer frame de la trayectoria (t = 0 ns)
        extract_frame_command = f"echo 1 | gmx trjconv -s {md_tpr_file} -f {os.path.join(analysis_dir, 'MD_center.xtc')} -o start.pdb -dump 0"
        subprocess.run(extract_frame_command, shell=True, check=True)
        logging.info(f"Frame inicial extraído y guardado en start.pdb en {folder}.")
        shutil.move("start.pdb", os.path.join(analysis_dir, "start.pdb"))

        # Calcular RMSD del ligando respecto a la proteína (usando números de grupo 1 para proteína y 13 para ligando)
        rmsd_command = f"echo 1 13 | gmx rms -s {md_tpr_file} -f {os.path.join(analysis_dir, 'MD_center.xtc')} -n {index_file} -tu ns -o rmsd_lig_prot.xvg"
        subprocess.run(rmsd_command, shell=True, check=True)
        logging.info(f"RMSD del ligando respecto a la proteína calculado y guardado en rmsd_lig_prot.xvg en {folder}.")
        shutil.move("rmsd_lig_prot.xvg", os.path.join(analysis_dir, "rmsd_lig_prot.xvg"))

        # Calcular RMSD del backbone de la proteína (grupo 4 es normalmente el backbone en GROMACS)
        rmsd_backbone_command = f"echo 4 4 | gmx rms -s {md_tpr_file} -f {os.path.join(analysis_dir, 'MD_center.xtc')} -n {index_file} -tu ns -o rmsd_backbone.xvg"
        subprocess.run(rmsd_backbone_command, shell=True, check=True)
        logging.info(f"RMSD del backbone de la proteína calculado y guardado en rmsd_backbone.xvg en {folder}.")
        shutil.move("rmsd_backbone.xvg", os.path.join(analysis_dir, "rmsd_backbone.xvg"))

        # Calcular RMSF (opcional)
        rmsf_command = f"echo 4 | gmx rmsf -s {md_tpr_file} -f {os.path.join(analysis_dir, 'MD_center.xtc')} -o rmsf.xvg"
        subprocess.run(rmsf_command, shell=True, check=True)
        logging.info(f"RMSF calculado y guardado en rmsf.xvg en {folder}.")
        shutil.move("rmsf.xvg", os.path.join(analysis_dir, "rmsf.xvg"))

        # Calcular enlaces de hidrógeno entre ligando y proteína seleccionando los grupos separadamente
        hbond_command = f"gmx hbond -s {md_tpr_file} -f {os.path.join(analysis_dir, 'MD_center.xtc')} -n {index_file} -num hb.xvg -tu ns"
        hbond_process = subprocess.Popen(hbond_command, shell=True, stdin=subprocess.PIPE)
        hbond_process.communicate(input=b"1\n13\n")  # Selección de grupos 1 (Proteína) y 13 (Ligando)
        hbond_process.wait()

        logging.info(f"Enlaces de hidrógeno calculados y guardados en hb.xvg en {folder}.")
        shutil.move("hb.xvg", os.path.join(analysis_dir, "hb.xvg"))

        # Calcular radio de giro
        gyrate_command = f"echo 1 | gmx gyrate -s {md_tpr_file} -f {os.path.join(analysis_dir, 'MD_center.xtc')} -o gyrate1.xvg"
        subprocess.run(gyrate_command, shell=True, check=True)
        logging.info(f"Radio de giro calculado y guardado en gyrate1.xvg en {folder}.")
        shutil.move("gyrate1.xvg", os.path.join(analysis_dir, "gyrate1.xvg"))

        # Calcular energía seleccionando términos "Potential" y "Total-Energy"
        energy_command = f"gmx energy -f {md_edr_file} -o energy1.xvg"
        energy_process = subprocess.Popen(energy_command, shell=True, stdin=subprocess.PIPE)
        energy_process.communicate(input=b"12\n14\n0\n")  # Selección de términos de energía: "Potential" (12) y "Total-Energy" (14)
        energy_process.wait()

        logging.info(f"Energía calculada y guardada en energy1.xvg en {folder}.")
        shutil.move("energy1.xvg", os.path.join(analysis_dir, "energy1.xvg"))

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al ejecutar los comandos en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado en {folder}: {e}")

# Función principal para ejecutar el proceso completo en todas las carpetas
def run_mds_analysis_in_all_folders():
    for folder in protein_folders:
        try:
            # Verificar si la carpeta existe
            if not os.path.exists(os.path.join(base_path, folder)):
                logging.warning(f"La carpeta {folder} no existe. Saltando...")
                continue

            os.chdir(os.path.join(base_path, folder))
            logging.info(f"Iniciando análisis de MDS en {folder}.")
            run_mds_analysis(folder)
            os.chdir(base_path)

        except Exception as e:
            logging.error(f"Error en el proceso de análisis de MDS en {folder}: {e}")
            os.chdir(base_path)

if __name__ == "__main__":
    run_mds_analysis_in_all_folders()


In [ ]:
# RMSD 
import os
import subprocess
import logging
import shutil

# Configurar el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Base path de las carpetas de proteínas
base_path = "/home/cruz/Escritorio/MDS_Manual_3/"

# Lista de carpetas de proteínas generadas anteriormente
protein_folders = [
    "E5_15_P-C-L_output", "E5_21_P-C-L_output", "E5_25_P-C-L_output", "E5_54_P-C-L_output",
    "E5_16_P-C-L_output", "E5_22_P-C-L_output", "E5_30_P-C-L_output", "E5_55_P-C-L_output",
    "E5_1_P-C-L_output",  "E5_24_P-C-L_output", "E5_47_P-C-L_output", "E5_8_P-C-L_output"
]

# Función para ejecutar el proceso de análisis de MDS
def run_mds_analysis(folder):
    try:
        # Definir la ruta donde están los archivos de la simulación usando rutas absolutas
        analysis_dir = os.path.join(base_path, folder, "analisis")
        
        # Verificar si los archivos necesarios existen dentro de la carpeta "analisis"
        md_tpr_file = os.path.join(analysis_dir, "MD.tpr")
        md_xtc_file = os.path.join(analysis_dir, "MD.xtc")
        md_edr_file = os.path.join(analysis_dir, "MD.edr")
        index_file = os.path.join(base_path, folder, "index.ndx")  # El archivo index.ndx está en la carpeta principal de la proteína

        # Imprimir las rutas de los archivos que se están buscando
        print(f"Buscando archivo TPR en: {md_tpr_file}")
        print(f"Buscando archivo XTC en: {md_xtc_file}")
        print(f"Buscando archivo EDR en: {md_edr_file}")
        print(f"Buscando archivo INDEX en: {index_file}")

        # Verificar la existencia de cada archivo
        if not os.path.exists(md_tpr_file):
            logging.warning(f"Archivo {md_tpr_file} no encontrado.")
        if not os.path.exists(md_xtc_file):
            logging.warning(f"Archivo {md_xtc_file} no encontrado.")
        if not os.path.exists(md_edr_file):
            logging.warning(f"Archivo {md_edr_file} no encontrado.")
        if not os.path.exists(index_file):
            logging.warning(f"Archivo {index_file} no encontrado.")

        # Si alguno de los archivos no existe, saltar la carpeta
        if not (os.path.exists(md_tpr_file) and os.path.exists(md_xtc_file) and os.path.exists(md_edr_file) and os.path.exists(index_file)):
            logging.warning(f"Archivos necesarios faltan en {folder}. Saltando...")
            return

        # Recentrar y reajustar coordenadas
        recenter_command = f"echo 1 0 | gmx trjconv -s {md_tpr_file} -f {md_xtc_file} -o MD_center.xtc -center -pbc mol -ur compact"
        subprocess.run(recenter_command, shell=True, check=True)
        logging.info(f"Coordenadas recentradas y guardadas en MD_center.xtc en {folder}.")
        shutil.move("MD_center.xtc", os.path.join(analysis_dir, "MD_center.xtc"))

        # Extraer el primer frame de la trayectoria (t = 0 ns)
        extract_frame_command = f"echo 1 | gmx trjconv -s {md_tpr_file} -f {os.path.join(analysis_dir, 'MD_center.xtc')} -o start.pdb -dump 0"
        subprocess.run(extract_frame_command, shell=True, check=True)
        logging.info(f"Frame inicial extraído y guardado en start.pdb en {folder}.")
        shutil.move("start.pdb", os.path.join(analysis_dir, "start.pdb"))

        # Calcular RMSD del ligando respecto a la proteína usando los grupos correctos del índice
        rmsd_command = f"echo Protein LIG | gmx rms -s {md_tpr_file} -f {os.path.join(analysis_dir, 'MD_center.xtc')} -n {index_file} -tu ns -o rmsd_lig_prot.xvg"
        subprocess.run(rmsd_command, shell=True, check=True)
        logging.info(f"RMSD del ligando respecto a la proteína calculado y guardado en rmsd_lig_prot.xvg en {folder}.")
        shutil.move("rmsd_lig_prot.xvg", os.path.join(analysis_dir, "rmsd_lig_prot.xvg"))

    except subprocess.CalledProcessError as e:
        logging.error(f"Error al ejecutar los comandos en {folder}: {e}")
    except Exception as e:
        logging.error(f"Error inesperado en {folder}: {e}")

# Función principal para ejecutar el proceso completo en todas las carpetas
def run_mds_analysis_in_all_folders():
    for folder in protein_folders:
        try:
            if not os.path.exists(os.path.join(base_path, folder)):
                logging.warning(f"La carpeta {folder} no existe. Saltando...")
                continue

            os.chdir(os.path.join(base_path, folder))
            logging.info(f"Iniciando análisis de MDS en {folder}.")
            run_mds_analysis(folder)
            os.chdir(base_path)

        except Exception as e:
            logging.error(f"Error en el proceso de análisis de MDS en {folder}: {e}")
            os.chdir(base_path)

if __name__ == "__main__":
    run_mds_analysis_in_all_folders()


In [ ]:
# Importar librerías necesarias
import os
import logging

# Ruta base para las carpetas de proteínas y datos
base_path = "/home/cruz/Escritorio/MDS_Manual_3/analysis_comparison/"

# Configurar los logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Función para cargar datos de RMSD desde un archivo de texto e imprimir su contenido
def load_rmsd_data(rmsd_file):
    """Carga datos de RMSD desde un archivo txt e imprime las líneas para depuración."""
    data = {}
    protein = None
    times = []
    values = []

    with open(rmsd_file, 'r') as f:
        logging.info(f"Leyendo el archivo: {rmsd_file}")
        for line in f:
            # Imprimir cada línea del archivo para inspección
            print(f"Línea del archivo: {line.strip()}")
            
            if line.startswith("# Datos para"):
                if protein and times and values:
                    logging.info(f"Cargando datos de {protein}: {len(times)} puntos")
                    data[protein] = (times, values)
                protein = line.split()[-1]
                times = []
                values = []
            elif line.strip() and not line.startswith("#") and "Time(ns)" not in line:
                try:
                    time, value = map(float, line.split())
                    times.append(time)
                    values.append(value)
                except ValueError:
                    logging.warning(f"Skipping invalid line: {line.strip()}")
        
        # Añadir últimos datos si existen
        if protein and times and values:
            logging.info(f"Cargando datos de {protein}: {len(times)} puntos")
            data[protein] = (times, values)
    
    # Verificar si se cargaron datos
    if not data:
        logging.error("No se pudieron cargar datos del archivo de RMSD.")
    return data

# Cargar los datos de RMSD
rmsd_file = os.path.join(base_path, "RMSD_lig_prot_data.txt")
rmsd_data = load_rmsd_data(rmsd_file)

# Si no hay datos, salir del script
if not rmsd_data:
    logging.error("No valid data available for analysis.")


In [ ]:
# Importar librerías necesarias
import os
import logging
import pandas as pd

# Ruta base para las carpetas de proteínas y datos
base_path = "/home/cruz/Escritorio/MDS_Manual_3/analysis_comparison/"

# Configurar los logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Función para cargar datos de RMSD desde un archivo de texto
def load_rmsd_data(rmsd_file):
    """Carga datos de RMSD desde un archivo txt e ignora encabezados innecesarios."""
    data = {}
    protein = None
    times = []
    values = []

    with open(rmsd_file, 'r') as f:
        logging.info(f"Leyendo el archivo: {rmsd_file}")
        for line in f:
            # Ignorar encabezados o comentarios
            if line.startswith("#") or "Time(ns)" in line:
                continue
            
            # Si es una línea con datos
            if line.strip():
                try:
                    time, value = map(float, line.split())
                    times.append(time)
                    values.append(value)
                except ValueError:
                    logging.warning(f"Skipping invalid line: {line.strip()}")
        
        # Añadir últimos datos si existen
        if times and values:
            logging.info(f"Datos cargados: {len(times)} puntos")
            data['RMSD'] = (times, values)
    
    # Verificar si se cargaron datos
    if not data:
        logging.error("No se pudieron cargar datos del archivo de RMSD.")
    return data

# Cargar los datos de RMSD
rmsd_file = os.path.join(base_path, "RMSD_lig_prot_data.txt")
rmsd_data = load_rmsd_data(rmsd_file)

# Si no hay datos, salir del script
if not rmsd_data:
    logging.error("No valid data available for analysis.")
else:
    # Convertir los datos a DataFrame
    df = pd.DataFrame({'Time (ns)': rmsd_data['RMSD'][0], 'RMSD (nm)': rmsd_data['RMSD'][1]})
    print(df.head())  # Mostrar las primeras filas para verificar


In [ ]:
#RMSD comparative analisis

import os
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.neighbors import LocalOutlierFactor

# Configurar los logs para guardarlos en un archivo
log_file = "/home/cruz/Escritorio/MDS_Manual_3/analysis_comparison/logs_rmsd_analysis.log"
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Ruta base para las carpetas de proteínas y datos
base_path = "/home/cruz/Escritorio/MDS_Manual_3/analysis_comparison/"
rmsd_file = os.path.join(base_path, "RMSD_lig_prot_data.txt")
statistics_file = os.path.join(base_path, "statistics_rmsd.txt")

# Función para cargar los datos de RMSD
def load_rmsd_data(rmsd_file):
    data = {}
    protein_name = None
    times = []
    values = []
    
    with open(rmsd_file, 'r') as f:
        for line in f:
            if "Time(ns)" in line:  # Ignorar la línea que contiene las etiquetas
                continue
            if line.startswith("# Data for"):
                if protein_name and times and values:
                    data[protein_name] = (times, values)
                protein_name = line.split()[-1]
                times = []
                values = []
            elif line.strip() and not line.startswith("#"):
                try:
                    time, value = map(float, line.split())
                    times.append(time)
                    values.append(value)
                except ValueError as e:
                    logging.warning(f"Error al procesar línea: {line.strip()}, error: {e}")
        if protein_name and times and values:
            data[protein_name] = (times, values)
    
    if not data:
        logging.error("No se pudieron cargar datos del archivo de RMSD.")
    return data

# Cargar los datos de RMSD
rmsd_data = load_rmsd_data(rmsd_file)

# Convertir los datos a DataFrame asegurando que todos los tiempos están alineados
if rmsd_data:
    # Crear una lista de tiempos combinada que abarque todos los tiempos presentes
    all_times = sorted(set(time for times, _ in rmsd_data.values() for time in times))
    
    # Crear el DataFrame con los tiempos combinados como índice
    df = pd.DataFrame(index=all_times)
    
    for protein, (times, values) in rmsd_data.items():
        # Crear un DataFrame temporal para cada proteína
        temp_df = pd.DataFrame({protein: values}, index=times)
        # Alinear los datos según todos los tiempos combinados, rellenando con NaN donde sea necesario
        df = df.join(temp_df, how="outer")

    df.index.name = "Time"
else:
    logging.error("No valid data available for analysis.")
    df = pd.DataFrame()

# Función para guardar resultados en un archivo de texto
def save_statistics_to_file(file_path, content):
    with open(file_path, 'a') as f:
        f.write(content + "\n")

# Función para obtener estadísticos descriptivos
def descriptive_statistics(df):
    mean = df.mean()
    std = df.std()
    stats_content = f"Mean RMSD:\n{mean}\n\nStandard Deviation RMSD:\n{std}\n"
    logging.info(stats_content)
    save_statistics_to_file(statistics_file, stats_content)
    return mean, std

# Función para realizar ANOVA
def perform_anova(df):
    df_melted = pd.melt(df.reset_index(), id_vars=['Time'], value_name='RMSD')
    df_melted.columns = ['Time', 'Protein', 'RMSD']
    
    model = ols('RMSD ~ Protein', data=df_melted).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    anova_content = f"ANOVA results:\n{anova_table}\n"
    logging.info(anova_content)
    save_statistics_to_file(statistics_file, anova_content)
    return anova_table

# Función para prueba de normalidad (Shapiro-Wilk)
def check_normality(df):
    normality_results = {}
    for column in df.columns:
        stat, p_value = stats.shapiro(df[column].dropna())
        normality_results[column] = (stat, p_value)
        normality_content = f"Normality test for {column}: W={stat}, p-value={p_value}\n"
        logging.info(normality_content)
        save_statistics_to_file(statistics_file, normality_content)
    return normality_results

# Función para detectar outliers usando Local Outlier Factor
def detect_outliers(df):
    lof = LocalOutlierFactor()
    outlier_flags = lof.fit_predict(df.fillna(0))
    outliers = df[outlier_flags == -1]
    outliers_content = f"Outliers detected:\n{outliers}\n"
    logging.info(outliers_content)
    save_statistics_to_file(statistics_file, outliers_content)
    return outliers

# Función para graficar el RMSD
def plot_rmsd(df):
    plt.figure(figsize=(10, 6))
    for column in df.columns:
        plt.plot(df.index, df[column], label=column)
    plt.xlabel("Time (ns)")
    plt.ylabel("RMSD (nm)")
    plt.title("RMSD Ligand-Protein Comparison")
    plt.legend()
    plt.grid(True)
    output_path = os.path.join(base_path, "RMSD_lig_prot_plot.png")
    plt.savefig(output_path)
    plt.close()
    logging.info(f"Gráfico guardado en {output_path}")

# Realizar análisis y guardar resultados
if not df.empty:
    # Iniciar archivo de resultados
    with open(statistics_file, 'w') as f:
        f.write("# Statistical Analysis Results (RMSD Ligand-Protein)\n\n")
    
    mean, std = descriptive_statistics(df)
    anova_results = perform_anova(df)
    normality_results = check_normality(df)
    outliers = detect_outliers(df)

    # Graficar los resultados de RMSD
    plot_rmsd(df)
else:
    logging.error("No data available for analysis.")

